In [1]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
from glob import glob
from scipy.ndimage import zoom
from torch.utils.data import Dataset,DataLoader
import albumentations as A
import torch
import torch.nn as nn
import timm
from sklearn.model_selection import KFold
import math
import os

In [2]:
path = "/kaggle/input/rsna2024-lsdc-part-1-eda-and-preparing-data"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
comp_dir = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification"
image_dir = f"{path}/cvt_png"
pd.set_option('future.no_silent_downcasting', True)

# Part 2.1 Dataset and Augs

In [5]:
train_y = pd.read_csv(f'{comp_dir}/train.csv')
study_ids = [int(study_id) for study_id in os.listdir(image_dir)]
train_y = train_y[train_y['study_id'].isin(study_ids)].reset_index(drop=True)
mapping = {"Normal/Mild":0,"Moderate":1,"Severe":2}
train_y = train_y.replace(mapping)

In [6]:
class RSNADataset(Dataset):
    def __init__(self,df:pd.DataFrame, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        study_id = self.df.iloc[idx,0]
        label = self.df.iloc[idx,1:].values.astype(np.int64)
        
        ret_image = []
        
        parent_path = f"{path}/cvt_png/{study_id}/Axial T2/"
        if len(os.listdir(parent_path))<20:
            image = [cv2.imread(image_dir,-1) for image_dir in glob(f"{parent_path}**")]
            image = np.transpose(np.stack(image),(1,2,0)).astype(np.uint8)
            image = zoom(image,(1,1,20/image.shape[2]))
        else:
            image = [cv2.imread(image_dir,-1) for image_dir in glob(f"{parent_path}**")]
            image = np.transpose(np.stack(image),(1,2,0)).astype(np.uint8)
        ret_image.append(image)    
        
        parent_path = f"{path}/cvt_png/{study_id}/Sagittal T1/"
        if len(os.listdir(parent_path))<10:
            image = [cv2.imread(image_dir,-1) for image_dir in glob(f"{parent_path}**")]
            image = np.transpose(np.stack(image),(1,2,0)).astype(np.uint8)
            image = zoom(image,(1,1,10/image.shape[2]))
        else:
            image = [cv2.imread(image_dir,-1) for image_dir in glob(f"{parent_path}**")]
            image = np.transpose(np.stack(image),(1,2,0)).astype(np.uint8)
        ret_image.append(image)
        
        parent_path = f"{path}/cvt_png/{study_id}/Sagittal T2/"
        if len(os.listdir(parent_path))<10:
            image = [cv2.imread(image_dir,-1) for image_dir in glob(f"{parent_path}**")]
            image = np.transpose(np.stack(image),(1,2,0)).astype(np.uint8)
            image = zoom(image,(1,1,10/image.shape[2]))
        else:
            image = [cv2.imread(image_dir,-1) for image_dir in glob(f"{parent_path}**")]
            image = np.transpose(np.stack(image),(1,2,0)).astype(np.uint8)
        ret_image.append(image)

        ret_image = np.concatenate(ret_image, axis=2)
        
        if self.transform is not None:
            image = self.transform(image=ret_image)['image']
        
        ret_image = ret_image.transpose(2,0,1)
        
        return ret_image, label

In [7]:
# Test
ret, label  = RSNADataset(train_y).__getitem__(1)
print(ret.shape)
del ret, label

(40, 224, 224)


In [8]:
transforms_train = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, border_mode=0, p=0.7),
    A.RandomBrightnessContrast(p=0.7),
    A.OneOf([
        A.MotionBlur(blur_limit=5),
        A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),
        A.GaussNoise(var_limit=(5.0, 30.0)),
    ], p=0.7),
    A.OneOf([
        A.OpticalDistortion(),
        A.GridDistortion(),
        A.ElasticTransform(),
    ], p=0.7),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, min_holes=1, min_height=8, min_width=8, p=0.5),
#     A.CLAHE(clip_limit=4.0,p=0.7),
    A.Resize(224,224),
    A.Normalize(mean=0.5,std=0.5)
])

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/blur/transforms.py:180: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [9]:
if not os.path.exists('rsna24-results'):
    os.makedirs('rsna24-results')       

# Part 2.2 Model

In [10]:
class RSNAModel(nn.Module):
    def __init__(self,in_c,n_classes):
        super(RSNAModel,self).__init__()
        self.encoder = timm.create_model("resnet18",
                                        in_chans = in_c,
                                        num_classes = 0,
                                        pretrained=False)
        self.encoder.fc = nn.Identity()
        
        self.lstm = nn.LSTM(512, 256, 3, bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512,256),
            nn.Dropout(0.3),
            nn.LeakyReLU(0.1),
            nn.Linear(256,n_classes)
        )        
    def forward(self,x):
        x = self.encoder(x)
        x, _ = self.lstm(x)
        x = self.head(x)
        return x

In [11]:
# Gradient Scaler
scaler = torch.cuda.amp.GradScaler(init_scale=4096)

# K Fold Cross Validation
skf = KFold(n_splits=4, shuffle=True, random_state=8)

# Weights for training
weights = torch.tensor([1.0,3.0,10.0])

num_epochs = 40

# Part 2.3 Training

In [12]:
train_y_len = len(train_y)
for fold, (trn_idx,val_idx) in enumerate(skf.split(range(train_y_len))):
    model = RSNAModel(40,75)
    model = model.to(device)
    
    dataset = RSNADataset(train_y.iloc[trn_idx],
                          transform=transforms_train)
    dataloader = DataLoader(dataset,
                            batch_size=4,
                            shuffle=False,
                            pin_memory=True,
                            drop_last = False,
                            num_workers=0)
    
    v_dataset = RSNADataset(train_y.iloc[val_idx],
                            transform=transforms_train)
    v_dataloader = DataLoader(v_dataset, batch_size=1,
                              shuffle=False,
                              pin_memory=True,
                              drop_last = False,
                              num_workers=0)
    
    n_labels = 25
    
    optimizer = torch.optim.AdamW(model.parameters(),lr=0.01)
    criterion_train = nn.CrossEntropyLoss(weight=weights.to(device))
    criterion_val = nn.CrossEntropyLoss(weight=weights)
    
    best_loss = 10 
    best_wll = 10
    es_step = 0

    for epoch in range(1,1+num_epochs):
        model.train()
        total_loss = 0 
        with tqdm(dataloader, leave=True) as pbar:
            optimizer.zero_grad()
            for idx, (x,t) in enumerate(pbar):
                x = x.to(device).float()
                t = t.to(device).long()
                loss = 0
                y = model(x)
                for col in range(n_labels):
                    pred = y[:,col*3:col*3+3]
                    gt = t[:,col]
                    loss = loss + criterion_train(pred,gt)/n_labels
                total_loss += loss.item()

                if not math.isfinite(loss):
                    print(f"Loss is {loss}, stopping training")
                    sys.exit(1)

                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1e9)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
        train_loss = total_loss/len(dataloader)
        print(f'train_loss: {train_loss:.6f}')

        total_loss = 0
        y_preds = []
        labels = []

        model.eval()
        with tqdm(v_dataloader,leave=True) as pbar:
            with torch.no_grad():
                for idx, (x,t) in enumerate(pbar):
                    x = x.to(device).float()
                    t = t.to(device).long()
                    loss = 0
                    y = model(x)
                    for col in range(n_labels):
                        pred = y[:, col*3:col*3+3]
                        gt = t[:,col]
                        loss = loss + criterion_train(pred,gt)/n_labels
                        y_preds.append(pred.float().cpu())
                        labels.append(gt.cpu())
                    total_loss += loss.item()
        val_loss = total_loss / len(v_dataloader)
        y_preds = torch.cat(y_preds,dim=0)
        labels = torch.cat(labels,dim=0)
        val_wll = criterion_val(y_preds,labels)

        print(f'val_loss: {val_loss:.6f}, val_wll: {val_wll:.6f}')
        if val_loss < best_loss or val_wll<best_wll:
            es_step = 0
            if val_loss<best_loss:
                print(f"epoch: {epoch}, best loss updated from {best_loss:.6f} to {val_loss:.6f}")
                best_loss = val_loss
            if val_wll<best_wll:
                print(f"epoch: {epoch}, best wll_metric updated from {best_wll:.6f} to {val_wll:.6f}")
                best_wll = val_wll
            fname = f'rsna24-results/best_wll_model_fold-{fold}.pt'
            torch.save(model.state_dict(),fname)                
        else:
            es_step += 1
            if es_step >= 3:
                print("early stopping")
                break

100%|██████████| 336/336 [08:06<00:00,  1.45s/it]


train_loss: 0.850750


100%|██████████| 448/448 [02:49<00:00,  2.64it/s]


val_loss: 0.666255, val_wll: 1.038749
epoch: 1, best loss updated from 10.000000 to 0.666255
epoch: 1, best wll_metric updated from 10.000000 to 1.038749


100%|██████████| 336/336 [03:03<00:00,  1.83it/s]


...

val_loss: 0.647151, val_wll: 1.096557


100%|██████████| 336/336 [03:09<00:00,  1.77it/s]


train_loss: 0.807492


100%|██████████| 447/447 [01:04<00:00,  6.97it/s]

val_loss: 0.651283, val_wll: 1.090462
early stopping
